---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [16]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df[72]

'Lithium 0.25 (7/11/77).  LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherwise neg.  TSH 3.28.  BUN/Cr: 16/0.83.  Lipids unremarkable.  B12 363, Folate >20.  CBC: 4.9/36/308 Pertinent Medical Review of Systems Constitutional:\n'

In [17]:
import re
import dateutil
def date_sorter():
    date_list = []
    a = df.str.findall(r'\d{1,2}\-\d{1,2}\-\d{2}|19\d{2}|20\d{2}|\d{1,2}\/\d{4}|\d{1,2}\/+\d+\/+\d{2,}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\,?\.?\-?\s?\d*\-?\,?\s?\d{4}|\d{2}\s?[A-Za-z]{3,10}\,?\.?\s?\d{4}|\w{3,}\s?2\d{1}\w+\,?\s?\d{4}|(?:January|February|March|April|May|June|July|August|September|October|November|December)\,?\s?\d*\s?\d{4}|\d{1,2}\-\d{1,2}\-\d{2}|19\d{2}|20\d{2}|\d{1,2}\/\d{4}|\d{1,2}\/+\d+\/+\d{2,}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.?\-?\s?\d{2}\-?\,?\s?\d{4}|\d{2}\s?[A-Za-z]{3,10}\,?\.?\s?\d{4}|\w{3,}\s?2\d{1}\w+\,?\s?\d{4}|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2}?\s?\d{4}|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d*\,?\s?\d{4}')
    for i, cc in enumerate(a):
        for b in cc:
            if b:
                if i < 20:
                    print(i, b)
                c = b.strip()
                if len(c) == 4:
                    c = '1/1/' + c
                elif re.findall(r'^\d{1,2}\/+\d{4}', c):
                    d = c.split('/')
                    c = d[0] + '/1/' + d[1]
                elif re.findall(r'^[A-Za-z]+\,?\s\d{4}', c):
                    c = '1 ' + c
                try:
                    date_list.append(dateutil.parser.parse(c))
                except:
                    continue
    df1 = pd.Series(date_list)
    df2 = df1.to_frame()
    df2.rename(columns={ df2.columns[0]: "tahir_data" }, inplace = True)
    print(df2)
    df2.sort_values(by = 'tahir_data', inplace=True)
    df2.reset_index(inplace = True) 
    series = pd.Series(df2['index'], dtype="int32")
    return series
date_sorter()

0 03/25/93
1 6/18/85
2 7/8/71
3 9/27/75
4 2/6/96
5 7/06/79
6 5/18/78
7 10/24/89
8 3/7/86
9 4/10/71
10 5/11/85
11 4/09/75
12 8/01/98
13 1/26/72
14 5/24/1990
15 1/25/2011
16 4/12/82
17 10/13/1976
18 4/24/98
19 5/21/77
    tahir_data
0   1993-03-25
1   1985-06-18
2   1971-07-08
3   1975-09-27
4   1996-02-06
5   1979-07-06
6   1978-05-18
7   1989-10-24
8   1986-03-07
9   1971-04-10
10  1985-05-11
11  1975-04-09
12  1998-08-01
13  1972-01-26
14  1990-05-24
15  2011-01-25
16  1982-04-12
17  1976-10-13
18  1998-04-24
19  1977-05-21
20  1998-07-21
21  1979-10-21
22  1990-03-03
23  1976-02-11
24  1984-07-25
25  1982-04-13
26  1989-09-22
27  1976-09-02
28  1971-09-12
29  1986-10-24
..         ...
470 1983-01-01
471 1999-01-01
472 2010-01-01
473 1975-01-01
474 1972-01-01
475 2015-01-01
476 1989-01-01
477 1994-01-01
478 1993-01-01
479 1996-01-01
480 2013-01-01
481 1974-01-01
482 1990-01-01
483 1995-01-01
484 2004-01-01
485 1987-01-01
486 1973-01-01
487 1992-01-01
488 1977-01-01
489 1985-01-01
490 

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    198
478    286
479    480
480    431
481    279
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int32